In [1]:
import undetected_chromedriver as uc
import re
from datetime import datetime


In [2]:
url = 'https://www.aruodas.lt/butai-vilniuje-santariskese-dangerucio-g-naujas-pilnai-irengtas-butas-i-kuri-gali-1-3305167/'

In [3]:
def get_html(url):
    options = uc.ChromeOptions()
    options.add_argument('--headless')
    with uc.Chrome(options=options) as driver:
        driver.get(url)
        html = driver.page_source
    return html


In [18]:
def text_strip_list(element_list):
    return [element.text_content().strip() for element in element_list]

def while_replace(string):
    while '  ' in string:
        string = string.replace('  ', ' ')

    return string

In [5]:
source = get_html(url)

In [6]:
# parse html with lxml
from lxml import html

tree = html.fromstring(source)

In [7]:
tree.body.find_class('price-eur')[0].text.strip()

'199 000 €'

In [8]:
# class obj-details dt:value name, dd:value
tree.body.find_class('obj-details')[0].findall('dt')[-5].text

'\n                        Pastato energijos suvartojimo klasė:\n                    '

In [33]:
property_info_values = text_strip_list(tree.body.find_class('obj-details')[0].findall('dd'))
property_info_value_names = text_strip_list(tree.body.find_class('obj-details')[0].findall('dt'))

property_info_values = [i for i in property_info_values if i != '']
property_info_value_names = property_info_value_names[:len(property_info_values)]

property_info = dict(zip(property_info_value_names, property_info_values))
property_info['Misc'] = text_strip_list(tree.body.find_class('special-comma'))
property_info['Description'] = tree.findall('.//*[@id="collapsedText"]')[0].text_content().strip()
try:
    property_info['Phone'] = tree.find_class('phone_item_0')[0].text.strip()
    property_info['Broker'] = True
except:
    property_info['Phone'] = tree.find_class('phone')[0].text
    property_info['Broker'] = False

del(property_info['Ypatybės:'])
del(property_info['Papildomos patalpos:'])
del(property_info['Papildoma įranga:'])
del(property_info['Apsauga:'])

property_info['Address'] = tree.find_class('obj-header-text')[0].text_content().strip()
property_info['Address'] = re.split(', \d+ kamb', property_info['Address'])[0]

stats_names = text_strip_list(tree.find_class('obj-stats simple')[0].find('dl').findall('dt'))
stats_values = text_strip_list(tree.find_class('obj-stats simple')[0].find('dl').findall('dd'))

stats = dict(zip(stats_names, stats_values))
property_info.update(stats)

# add date scraped
property_info['Date_scraped'] = datetime.now().strftime("%d/%m/%Y %H:%M:%S")


{'Namo numeris:': '31',
 'Buto numeris:': '8',
 'Plotas:': '55,81 m²',
 'Kambarių sk.:': '2',
 'Aukštas:': '3',
 'Aukštų sk.:': '4',
 'Metai:': '2021',
 'Pastato tipas:': 'Mūrinis',
 'Šildymas:': 'Elektra, aeroterminis',
 'Įrengimas:': 'Įrengtas',
 'Pastato energijos suvartojimo klasė:': 'A++',
 'Misc': ['Aukštos lubos',
  'Nauja kanalizacija',
  'Nauja elektros instaliacija',
  'Uždaras kiemas',
  'Internetas',
  'Kabelinė televizija',
  'Virtuvė sujungta su kambariu',
  'Balkonas',
  'Kondicionierius',
  'Skalbimo mašina',
  'Su baldais',
  'Šaldytuvas',
  'Virtuvės komplektas',
  'Viryklė',
  'Plastikiniai vamzdžiai',
  'Indaplovė',
  'Dušo kabina',
  'Rekuperacinė sistema',
  'Šarvuotos durys',
  'Kodinė laiptinės spyna'],
 'Description': 'NAUJAS PILNAI ĮRENGTAS BUTAS, Į KURĮ GALI ĮSIKELTI JAU ŠIANDIEN!\n\nParduodamas jaukiai ir stilingai įrengtas 55.81 kv. m dviejų kambarių butas itin ekonomiškame A++ energinio naudingumo name, kurio langai orientuoti į pietų bei rytų puses su erd

In [38]:
stats_names = text_strip_list(tree.find_class('obj-stats simple')[0].find('dl').findall('dt'))
stats_values = text_strip_list(tree.find_class('obj-stats simple')[0].find('dl').findall('dd'))

stats = dict(zip(stats_names, stats_values))
property_info.update(stats)


In [40]:
property_info

{'Namo numeris:': '31',
 'Buto numeris:': '8',
 'Plotas:': '55,81 m²',
 'Kambarių sk.:': '2',
 'Aukštas:': '3',
 'Aukštų sk.:': '4',
 'Metai:': '2021',
 'Pastato tipas:': 'Mūrinis',
 'Šildymas:': 'Elektra, aeroterminis',
 'Įrengimas:': 'Įrengtas',
 'Pastato energijos suvartojimo klasė:': 'A++',
 'Misc': ['Aukštos lubos',
  'Nauja kanalizacija',
  'Nauja elektros instaliacija',
  'Uždaras kiemas',
  'Internetas',
  'Kabelinė televizija',
  'Virtuvė sujungta su kambariu',
  'Balkonas',
  'Kondicionierius',
  'Skalbimo mašina',
  'Su baldais',
  'Šaldytuvas',
  'Virtuvės komplektas',
  'Viryklė',
  'Plastikiniai vamzdžiai',
  'Indaplovė',
  'Dušo kabina',
  'Rekuperacinė sistema',
  'Šarvuotos durys',
  'Kodinė laiptinės spyna'],
 'Description': 'NAUJAS PILNAI ĮRENGTAS BUTAS, Į KURĮ GALI ĮSIKELTI JAU ŠIANDIEN!\n\nParduodamas jaukiai ir stilingai įrengtas 55.81 kv. m dviejų kambarių butas itin ekonomiškame A++ energinio naudingumo name, kurio langai orientuoti į pietų bei rytų puses su erd

In [52]:
property_info['Address'] = tree.find_class('obj-header-text')[0].text_content().strip()
property_info['Address'] = re.split(', \d+ kamb', property_info['Address'])[0]


In [71]:
photos = tree.find_class('link-obj-thumb')
photos = [photo.get('href') for photo in photos]
set(photos)


{None,
 'https://3darchitects.lt/santariskes/butas1/',
 'https://aruodas-img.dgn.lt/object_62_109150065/vilnius-santariskes-dangerucio-g-2.jpg',
 'https://aruodas-img.dgn.lt/object_62_109150067/vilnius-santariskes-dangerucio-g-2.jpg',
 'https://aruodas-img.dgn.lt/object_62_109150075/vilnius-santariskes-dangerucio-g-2.jpg',
 'https://aruodas-img.dgn.lt/object_62_109150081/vilnius-santariskes-dangerucio-g-2.jpg',
 'https://aruodas-img.dgn.lt/object_62_109150085/vilnius-santariskes-dangerucio-g-2.jpg',
 'https://aruodas-img.dgn.lt/object_62_109150087/vilnius-santariskes-dangerucio-g-2.jpg',
 'https://aruodas-img.dgn.lt/object_62_109150089/vilnius-santariskes-dangerucio-g-2.jpg',
 'https://aruodas-img.dgn.lt/object_62_109150093/vilnius-santariskes-dangerucio-g-2.jpg',
 'https://aruodas-img.dgn.lt/object_62_109150095/vilnius-santariskes-dangerucio-g-2.jpg',
 'https://aruodas-img.dgn.lt/object_62_109150099/vilnius-santariskes-dangerucio-g-2.jpg',
 'https://aruodas-img.dgn.lt/object_62_109150